## Prompt

In [35]:
from langchain import PromptTemplate

prompt = PromptTemplate(
    input_variables=["position", "info"], # 삭제 가능
    template="{position}의 {info}에 대해서 알려줘")
msg = prompt.format(position="왕십리", info="유래")
print(msg)

# 부분 변수로 초기화할 수 있다.
partial_template = prompt.partial(position="서울")
print("부분 초기화 : ", partial_template)
print("전체 : ", partial_template.format(info="위치"))

왕십리의 유래에 대해서 알려줘
부분 초기화 :  input_variables=['info'] output_parser=None partial_variables={'position': '서울'} template='{position}의 {info}에 대해서 알려줘' template_format='f-string' validate_template=True
전체 :  서울의 위치에 대해서 알려줘


### MessagePromptTemplate

In [27]:
from langchain.prompts import (
ChatPromptTemplate,
ChatMessagePromptTemplate,
SystemMessagePromptTemplate,
AIMessagePromptTemplate,
HumanMessagePromptTemplate
)

sys_prompt = SystemMessagePromptTemplate.from_template(
    "당신은 {input}을 {output}으로 번역해주는 봇입니다."
)
human_prompt = HumanMessagePromptTemplate.from_template("{text}")
chat_prompt = ChatPromptTemplate.from_messages([sys_prompt, human_prompt])
msg = chat_prompt.format_prompt(
    input="영어",
    output="한국어",
    text="I love programming"
).to_messages()
print(msg)

[SystemMessage(content='당신은 영어을 한국어으로 번역해주는 봇입니다.', additional_kwargs={}), HumanMessage(content='I love programming', additional_kwargs={}, example=False)]


In [21]:
chat_msg_prompt = ChatMessagePromptTemplate.from_template(
    role="Jedi",
    template="May the {subject} be with you"
)
chat_msg_prompt.format(subject="force")

ChatMessage(content='May the force be with you', additional_kwargs={}, role='Jedi')

In [25]:
from langchain.prompts import MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage

human_msg_template = HumanMessagePromptTemplate.from_template(
    "대화를 {count}자로 요약해줘"
)
chat_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="conversation"),
    human_msg_template
])
human_message = HumanMessage(content="프로그래밍을 배우는 가장 좋은 방법은 무엇인가요?")
ai_message = AIMessage(content="""\
1. 프로그래밍 언어를 선택합니다: 배우고 싶은 프로그래밍 언어를 결정합니다.
2. 기본부터 시작하세요: 변수, 데이터 유형, 제어 구조와 같은 기본 프로그래밍 개념을 익힙니다.
3. 연습: 프로그래밍을 배우는 가장 좋은 방법은 실습을 통한 경험입니다\.
""")

chat_prompt.format_prompt(conversation=[human_message, ai_message], count="10").to_messages()

[HumanMessage(content='프로그래밍을 배우는 가장 좋은 방법은 무엇인가요?', additional_kwargs={}, example=False),
 AIMessage(content='1. 프로그래밍 언어를 선택합니다: 배우고 싶은 프로그래밍 언어를 결정합니다.\n2. 기본부터 시작하세요: 변수, 데이터 유형, 제어 구조와 같은 기본 프로그래밍 개념을 익힙니다.\n3. 연습: 프로그래밍을 배우는 가장 좋은 방법은 실습을 통한 경험입니다\\.\n', additional_kwargs={}, example=False),
 HumanMessage(content='대화를 10자로 요약해줘', additional_kwargs={}, example=False)]

### 조합

In [36]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts import PromptTemplate

intro_prompt = PromptTemplate.from_template("당신은 {person}입니다.")
example_prompt = PromptTemplate.from_template("""
다음은 대화의 예입니다.
Q: {example_q}
A: {example_a}
""")
start_prompt = PromptTemplate.from_template("""
Q: {input}
A:""")

pipeline_prompt = PipelinePromptTemplate(
    final_prompt=PromptTemplate.from_template(
        """{introduction}
        {example}
        {start}
        """
    ),
    pipeline_prompts=[
        ("introduction", intro_prompt),
        ("example", example_prompt),
        ("start", start_prompt)
    ]
)
pipeline_prompt.input_variables

['example_a', 'person', 'input', 'example_q']

In [38]:
msg = pipeline_prompt.format(
    person="일론 머스크",
    example_q="당신이 제일 좋아하는 차는 뭔가요?",
    example_a="테슬라요",
    input="당신이 제일 좋아하는 SNS 서비스는 뭐죠?"
)
print(msg)

당신은 일론 머스크입니다.
        
다음은 대화의 예입니다.
Q: 당신이 제일 좋아하는 차는 뭔가요?
A: 테슬라요

        
Q: 당신이 제일 좋아하는 SNS 서비스는 뭐죠?
A:
        


### 파일로 프롬프트 로드하기

In [40]:
from langchain.prompts import load_prompt

prompt = load_prompt("./data/my_prompt.json")
print(prompt.input_variables)
prompt.format(author="아이작 아시모프", book="파운데이션")

['author', 'book']


'아이작 아시모프 작가의 파운데이션에 대해서 알려줘'